### General Idea
In this notebook, I will pre-process all the raw data. There will be 2 processing stages:
 - Stage 1 - General pre-processing:
     - Resample data to [0.5,0.5,1] voxel spacing.
     - Cut or pad x-y plane to 512x512
     - Cutting the intensity window to [-200,600]
     - Rescale to [0,255] values (needed for segmentation-models-3D)
     - Remove the slices that don't have vessel segmentations.
     - Pad (zero) in z-axis until nearest multiple of 64 (not neeeded with the new random sampler but doesnt hurt)
     - Convert the 2 labels assigned for kidneys into just 1, and remove the extra classes.
         - Renal cavities were converted to kidneys here
     - After this, the classes are:
 
        0 - background
        1 - tumor
        2 - veins
        3 - arteries
        4 - kidneys
     - **This data will be saved as .nrrd files to folder /Andre/SAIAD_data_processed/**

In [1]:
from skimage import io
import numpy as np
import nrrd
import os
import glob
import sys
from utils.Visualization import *
from utils.Processing import *
from utils.Other import *

%matplotlib inline

%load_ext autoreload
%autoreload 2
data_path = '../SAIAD-project/Data/SAIAD_data_cleared/'

### Reading the data, resample to voxel size and cutting/padding x-y to 512x512

In [2]:
voxel_spacings = [0.5, 0.5, 1]
scans, segmentations, headers_scans, headers_segm, patient_names = read_data_and_resample(data_path, voxel_spacings, verbose=1)


Patient: SAIAD 7, Shape: (512, 512, 288)
Patient: SAIAD 9, Shape: (512, 512, 420)
Patient: SAIAD 1, Shape: (512, 512, 221)
Patient: SAIAD 2 Z BIS, Shape: (512, 512, 299)
Patient: SAIAD 14, Shape: (512, 512, 318)
Patient: SAIAD 13, Shape: (512, 512, 302)
Patient: SAIAD 20 BIS vasculaire, Shape: (512, 512, 641)
Patient: SAIAD 12, Shape: (512, 512, 192)
Patient: SAIAD 15, Shape: (512, 512, 316)
Patient: SAIAD 16 vasculaire, Shape: (512, 512, 300)
Patient: SAIAD 5, Shape: (512, 512, 429)
Patient: SAIAD 2, Shape: (512, 512, 402)
Patient: SAIAD 18 vasculaire, Shape: (512, 512, 375)
Patient: SAIAD 16 bis vasculaire, Shape: (512, 512, 274)
Patient: SAIAD 18 TER vasculaire, Shape: (512, 512, 162)
Patient: SAIAD 11, Shape: (512, 512, 450)
Patient: SAIAD 19 BIS vasculaire, Shape: (512, 512, 321)
Patient: TEST SAIAD 1, Shape: (512, 512, 221)
Patient: SAIAD 18 BIS vasculaire, Shape: (512, 512, 250)
Patient: SAIAD 17 BIS vasculaire, Shape: (512, 512, 272)
Patient: SAIAD 17 vasculaire, Shape: (512, 5

### Stage 1 - General pre-processing

In [3]:
scans_proc, segms_proc = process_data_saiad(scans, segmentations, patient_names, verbose=1)

Processing scan  SAIAD 7 , Shape: (512, 512, 128)
Processing scan  SAIAD 9 , Shape: (512, 512, 192)
Processing scan  SAIAD 1 , Shape: (512, 512, 128)
Processing scan  SAIAD 2 Z BIS , Shape: (512, 512, 128)
Processing scan  SAIAD 14 , Shape: (512, 512, 192)
Processing scan  SAIAD 13 , Shape: (512, 512, 192)
Processing scan  SAIAD 20 BIS vasculaire , Shape: (512, 512, 192)
Processing scan  SAIAD 12 , Shape: (512, 512, 128)
Processing scan  SAIAD 15 , Shape: (512, 512, 192)
Processing scan  SAIAD 16 vasculaire , Shape: (512, 512, 128)
Processing scan  SAIAD 5 , Shape: (512, 512, 256)
Processing scan  SAIAD 2 , Shape: (512, 512, 128)
Processing scan  SAIAD 18 vasculaire , Shape: (512, 512, 128)
Processing scan  SAIAD 16 bis vasculaire , Shape: (512, 512, 192)
Processing scan  SAIAD 18 TER vasculaire , Shape: (512, 512, 128)
Processing scan  SAIAD 11 , Shape: (512, 512, 192)
Processing scan  SAIAD 19 BIS vasculaire , Shape: (512, 512, 128)
Processing scan  TEST SAIAD 1 , Shape: (512, 512, 1

In [10]:
# Save to .nrrd files

stage1_folder = '../SAIAD-project/Data/SAIAD_data_processed_TEST/'###
os.mkdir(stage1_folder)

# Get names
file_names = []
for name in patient_names:
    file_names.append(name.split("/")[-1])
    
# Save to files
for i in range(len(scans)):
    
    # Make folder
    patient_folder = stage1_folder + file_names[i]
    os.mkdir(patient_folder)
    
    # Save scans and segmentations
    nrrd.write(patient_folder + '/scan.nrrd', scans[i], headers_scans[i])
    nrrd.write(patient_folder + '/segm.nrrd', segmentations[i], headers_segm[i])

    

In [12]:
# Test reading data
pat, header = nrrd.read(patient_folder + '/scan.nrrd')
segm, header = nrrd.read(patient_folder + '/segm.nrrd')
ImageSliceViewer3D(pat,segm)
print(header)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

OrderedDict([('type', 'uint8'), ('dimension', 3), ('space', 'left-posterior-superior'), ('sizes', array([512, 512, 128])), ('space directions', array([[0.5, 0. , 0. ],
       [0. , 0.5, 0. ],
       [0. , 0. , 1. ]])), ('kinds', ['domain', 'domain', 'domain']), ('encoding', 'raw'), ('space origin', array([-101.294, -180.294,  991.9  ]))])
